# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
houses = pd.read_csv("cal_housing_clean.csv")

In [3]:
houses.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
houses.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
label = houses["medianHouseValue"]

In [6]:
features = houses.drop("medianHouseValue", axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.33)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

In [10]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
houses.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [16]:
age = tf.feature_column.numeric_column("housingMedianAge")
rooms = tf.feature_column.numeric_column("totalRooms")
bedrooms = tf.feature_column.numeric_column("totalBedrooms")
population = tf.feature_column.numeric_column("population")
households = tf.feature_column.numeric_column("households")
income = tf.feature_column.numeric_column("medianIncome")

In [17]:
feat_cols = [age, rooms, bedrooms, population, households, income]

In [18]:
y_train.shape

(13828,)

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [20]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [86]:
model = tf.estimator.DNNRegressor(hidden_units=[6, 10, 10, 10], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\zEros\\AppData\\Local\\Temp\\tmpucxzmoxq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015DDBB58748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [87]:
model.train(input_func, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\zEros\AppData\Local\Temp\tmpucxzmoxq\model.ckpt.
INFO:tensorflow:loss = 264859880000.0, step = 1
INFO:tensorflow:global_step/sec: 387.135
INFO:tensorflow:loss = 138865740000.0, step = 101 (0.260 sec)
INFO:tensorflow:global_step/sec: 524.96
INFO:tensorflow:loss = 76643700000.0, step = 201 (0.189 sec)
INFO:tensorflow:global_step/sec: 547.904
INFO:tensorflow:loss = 61515186000.0, step = 301 (0.185 sec)
INFO:tensorflow:global_step/sec: 530.516
INFO:tensorflow:loss = 173820120000.0, step = 401 (0.184 sec)
INFO:tensorflow:global_step/sec: 569.698
INFO:tensorflow:loss = 128618380000.0, step = 501 (0.182 sec)
INFO:tensorflow:global_step/sec: 527.725
INFO:tensorflow:loss = 127286755000.0, step = 601 (0.186 s

INFO:tensorflow:global_step/sec: 529.282
INFO:tensorflow:loss = 104467554000.0, step = 7701 (0.191 sec)
INFO:tensorflow:global_step/sec: 530.54
INFO:tensorflow:loss = 48871680000.0, step = 7801 (0.188 sec)
INFO:tensorflow:global_step/sec: 516.917
INFO:tensorflow:loss = 39903420000.0, step = 7901 (0.195 sec)
INFO:tensorflow:global_step/sec: 506.035
INFO:tensorflow:loss = 147451610000.0, step = 8001 (0.197 sec)
INFO:tensorflow:global_step/sec: 501.536
INFO:tensorflow:loss = 134306940000.0, step = 8101 (0.198 sec)
INFO:tensorflow:global_step/sec: 527.601
INFO:tensorflow:loss = 92355300000.0, step = 8201 (0.191 sec)
INFO:tensorflow:global_step/sec: 512.895
INFO:tensorflow:loss = 92969960000.0, step = 8301 (0.193 sec)
INFO:tensorflow:global_step/sec: 493.997
INFO:tensorflow:loss = 32436445000.0, step = 8401 (0.201 sec)
INFO:tensorflow:global_step/sec: 525.643
INFO:tensorflow:loss = 66330976000.0, step = 8501 (0.192 sec)
INFO:tensorflow:global_step/sec: 488.83
INFO:tensorflow:loss = 77688610

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [88]:
predic_input_func = tf.estimator.inputs.pandas_input_fn(X_test, y_test, batch_size=10, num_epochs=1, shuffle=False)

In [89]:
results = model.predict(predic_input_func)

In [90]:
predictions = []

for pred in results:
    predictions.append(pred["predictions"])
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\zEros\AppData\Local\Temp\tmpucxzmoxq\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [91]:
from sklearn.metrics import mean_squared_error

In [92]:
mean_squared_error(y_test,predictions)**0.5

193292.78250151477

In [78]:
houses.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


# Great Job!